<a href="https://colab.research.google.com/github/shynybayev/NLP-Labs/blob/master/Lab2-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
!pip install  wikipedia


  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=17298013ff7d97c30dce84e1934a039571944ab487cf7137fafadc163ae9f05e
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import os
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json


In [7]:
import pandas as pd
import wikipedia

num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
from google.colab import drive 


2


In [8]:
!pip install yargy


     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 7.1MB 14.3MB/s 


In [0]:
import warnings
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)


In [11]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
import pandas as pd

CHARGE_NUMBER = 17

df = pd.read_csv("/content/gdrive/My Drive/lab2_words.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(5)


,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
384,NaN,385,17,Assan Shynybayev,ЖЕЛАТИН,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,NaN,386,17,Assan Shynybayev,ЖЕРЕХ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,NaN,387,17,Assan Shynybayev,ЖИВИЦА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,NaN,388,17,Assan Shynybayev,ЖИЛЕТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,NaN,389,17,Assan Shynybayev,ЖИМОЛОСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("-"*50)
    except: 
        pass


ЖЕЛАТИН
Желатин (фр. gelatine, от лат. gelatus «замороженныи»; менее распространенная форма: желатина) — бесцветныи или имеющии желтоватыи оттенок частично гидролизованныи белок коллаген, прозрачная вязкая масса, продукт переработки (денатурации) соединительнои ткани животных (коллагена). В зависимости от сырья и целеи применения имеет различные торговые названия: кроличии, мездровыи (кожныи), пергаментныи, костныи, рыбии (осетровыи), столярныи (костныи)  и т.д.
--------------------------------------------------
ЖЕРЕХ
Жерех, жерех, черех, белесть, белезна, красногубыи жерех, аральскии жерех или шереспер (лат. Aspius aspius) — самыи распространенныи вид рыб рода Жерехи семеиства карповых отряда карпообразных.
--------------------------------------------------
ЖИВИЦА
Живица (терпентин) — смолистая густая масса, выделяющаяся из разрезов на хвоиных деревьях. Застывает при выходе на поверхность, тем самым предохраняет древесину от проникновения короедов, грибов и других вредоносных влиянии,